In [75]:
ig_lendict={}
ig_lens=open("./data/igenome_lengths.txt").readlines()
print ig_lens[0:5]
for i in ig_lens[1:]:
    ig_lendict[i.split("\t")[0]]=i.split("\t")[1].replace("\n","")
ig_lendict

['phage\tgenome_len\n', '1.080.O.\t10046\n', '1.043.O.\t10272\n', '1.095.O.\t10436\n', '1.107.C.\t10447\n']


{'1.007.O.': '49244',
 '1.008.O.': '10579',
 '1.009.O.': '44443',
 '1.020.O.': '10636',
 '1.028.O.': '31704',
 '1.029.O.': '46513',
 '1.030.O.': '45827',
 '1.031.O.': '152942',
 '1.042.O.': '49730',
 '1.043.O.': '10272',
 '1.052.A.': '42889',
 '1.054.O.': '41774',
 '1.063.O.': '128641',
 '1.074.O.': '36922',
 '1.077.O.': '44047',
 '1.080.O.': '10046',
 '1.082.O.': '35810',
 '1.084.O.': '141906',
 '1.086.O.': '50835',
 '1.087.A.': '45674',
 '1.090.B.': '41868',
 '1.095.O.': '10436',
 '1.107.C.': '10447',
 '1.110.O.': '37556',
 '1.111.B.': '40209',
 '1.121.O.': '145590',
 '1.122.B.': '44523',
 '1.123.O.': '46071',
 '1.131.O.': '37933',
 '1.135.O.': '41918',
 '1.144.O.': '44418',
 '1.148.O.': '36789',
 '1.151.O.': '44307',
 '1.154.O.': '37136',
 '1.161.O.': '140668',
 '1.165.O.': '37046',
 '1.170.O.': '133692',
 '1.176.O.': '36463',
 '1.177.O.': '45439',
 '1.181.O.': '50228',
 '1.182.O.': '36910',
 '1.185.O.': '43397',
 '1.187.O.': '133254',
 '1.189.O.': '36855',
 '1.190.O.': '21785',
 '1

In [3]:
rl=open("./data/read_lengths.txt").readlines()

ldict={}

for l in rl:
    ldict[l.split("\t")[1]]=l.split("\t")[-1].replace("\n","")

In [21]:
from __future__ import division
fblast=open("./data/pb_subreads_vs_igenomes_concat_filt1.txt","w")

with open("./data/pb_subreads_vs_igenomes_concat.txt") as infile:
    for i in infile:
        vec=i.split("\t")
        aln_len=vec[3]
        read_len=int(ldict[vec[0]])
        qhitlen=int(vec[7])-int(vec[6])
        cov=qhitlen/read_len
        pctid=float(vec[2])
        if cov>0.99 and pctid>80:
            fblast.write(i)
    fblast.close()

In [8]:
%load_ext rpy2.ipython
%R library(ggplot2);library(magrittr); library(dplyr)

//anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘dplyr’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
//anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  res = super(Function, self).__call__(*new_args, **new_kwargs)
//anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  res = super(Function, self).__call__(*new_args, **new_kwargs)


array(['dplyr', 'magrittr', 'ggplot2', 'tools', 'stats', 'graphics',
       'grDevices', 'utils', 'datasets', 'methods', 'base'], 
      dtype='|S9')

In [37]:
%%R
fb=read.table("./data/pb_subreads_vs_igenomes_concat_filt1.txt", sep="\t")
nrow(fb)%>%paste("there are",.,"reads that have greater than 99% alignment and 80% ID to concatenated genomes", sep=" ")%>%print

[1] "there are 94921 reads that have greater than 99% alignment and 80% ID to concatenated genomes"


In [19]:
from __future__ import division
fblast=open("./data/pb_subreads_vs_igenomes_filt1.txt","w")

with open("./data/pb_subreads_vs_igenomes.txt") as infile:
    for i in infile:
        vec=i.split("\t")
        aln_len=vec[3]
        read_len=int(ldict[vec[0]])
        qhitlen=int(vec[7])-int(vec[6])
        cov=qhitlen/read_len
        pctid=float(vec[2])
        if cov>0.99 and pctid>85:
            fblast.write(i)
    fblast.close()

In [20]:
%%R
fb=read.table("./data/pb_subreads_vs_igenomes_filt1.txt", sep="\t")
nrow(fb)%>%paste("there are",.,"reads that have greater than 99% alignment and 80% ID to non-concatenated genomes", sep=" ")%>%print

[1] "there are 37441 reads that have greater than 99% alignment to non-concatenated genomes"


In [67]:
%%R
fb=read.table("./data/pb_subreads_vs_igenomes_concat_filt1.txt", sep="\t")

fb%>%group_by(V1, V2)%>%summarise(count=n())->read_ig_pairs


read_ig_pairs%>%nrow%>%print
read_ig_pairs$V1%>%unique%>%length%>%print
read_ig_pairs$V1%>%unique->rms
read_ig_pairs%>%group_by(count)%>%summarise(ct=n())
read_ig_pairs%>%group_by(V1)%>%summarise(ct=n())%>%group_by(ct)%>%summarise(ct1=n())->read_dist
colnames(read_dist)<-c("number_genomes_hit","read_count")
print(read_dist)
write.table(read_ig_pairs, "./data/concat_blast_read_ig_pairs.txt",sep="\t", quote=FALSE, row.names=FALSE)

In [42]:
%Rpull rms
tln=0
for r in rms:
    tln+=int(ldict[r])
print tln

219817239


In [68]:
pairs=open("./data/concat_blast_read_ig_pairs.txt").readlines()

In [71]:
from collections import defaultdict
read_dict=defaultdict(lambda: [])

for p in pairs[1:]:
    phage=p.split("\t")[1].split("_")[1]
    read=p.split("\t")[0]
    read_dict[phage]+=[read]

In [77]:
for p in read_dict.keys():
    reads=read_dict[p]
    genome_size=int(ig_lendict[p])
    length=0
    for r in reads:
        rlen=int(ldict[r])
        length+=rlen
    coverage=round(length/genome_size)
    print "coverage for %s is %s" % (p, coverage)
    

coverage for 1.030.O. is 37.0
coverage for 1.236.O. is 138.0
coverage for 1.257.O. is 28.0
coverage for 1.262.O. is 42.0
coverage for 1.009.O. is 36.0
coverage for 1.090.B. is 61.0
coverage for 1.161.O. is 50.0
coverage for 1.144.O. is 57.0
coverage for 1.054.O. is 41.0
coverage for 1.095.O. is 45.0
coverage for 1.215.A. is 51.0
coverage for 2.130.O. is 66.0
coverage for 1.193.O. is 42.0
coverage for 1.110.O. is 48.0
coverage for 1.121.O. is 83.0
coverage for 1.187.O. is 57.0
coverage for 1.042.O. is 51.0
coverage for 1.084.O. is 49.0
coverage for 1.043.O. is 40.0
coverage for 1.028.O. is 101.0
coverage for 1.189.O. is 58.0
coverage for 1.239.O. is 62.0
coverage for 1.276.O. is 55.0
coverage for 1.205.O. is 48.0
coverage for 1.190.O. is 30.0
coverage for 2.095.B. is 70.0
coverage for 2.117.O. is 24.0
coverage for 1.261.O. is 58.0
coverage for 1.182.O. is 48.0
coverage for 1.293.O. is 119.0
coverage for 1.209.O. is 24.0
coverage for 1.082.O. is 68.0
coverage for 1.131.O. is 66.0
coverag

### Below python script used to sort reads into fastq files based on genome-matching read bins on server.

In [ ]:
# %load ./scripts/extract_genome_read_bins.py
#!usr/bin/python
from collections import defaultdict
from Bio import SeqIO
import os

def write_ids_to_fastq(id_list, fastq_in, fastq_out):
    out_recs=[]
    for record in SeqIO.parse(open_fastq(fastq_in), "fastq"):
        if record.id in id_list:
            out_recs.append(record)
            
    out=open(fastq_out, "w")
    SeqIO.write(out_recs, out, "fastq")
    out.close()
    
def open_fastq(fastq):
    if fastq.endswith(".gz"):
        fastq=gzip.open(fastq, "rU")
    else:
        fastq=open(fastq, "rU")
    return fastq

            
os.mkdir("./binned_reads/")

pairs=open("concat_blast_read_ig_pairs.txt").readlines()

read_dict=defaultdict(lambda: [])    #create dictionary with phage as key, and reads that match phage as a list as the entry

for p in pairs[1:]:
    phage=p.split("\t")[1].split("_")[1]
    read=p.split("\t")[0]
    read_dict[phage]+=[read]            

for p in read_dict.keys():
    reads=set(read_dict[p])
    fastq_out="./binned_reads/%s_pb_subreads.fastq" % p
    write_ids_to_fastq(reads, "all_subreads.fastq", fastq_out)
    


#### looking into using a smaller alignment threshold to account for sequences potentially missed by the initial blast

In [80]:
# %load ./scripts/functions.py
#!usr/bin/python
from __future__ import division
from pyfaidx import Fasta

def write_read_lengths_to_file(read_fasta_files, output_file):
    out=open(output_file,"w")
    out.write("fasta_file\tseq_id\tread_len\n")
    
    readfiles=list(read_fasta_files)
    for r in readfiles:
        f=Fasta(r)
        for i in f.keys():
            length=len(str(f[i]))
            fasta=r.split("/")[-1]
            sequence=i
            out.write("%s\t%s\t%s\n" % (fasta, sequence, length))
    out.close()

def create_read_length_dict(dict_name, read_length_file):
    rl=open(read_length_file).readlines()
    ldict={}
    for l in rl:
        ldict[l.split("\t")[1]]=l.split("\t")[-1].replace("\n","")
    return ldict

def filter_reads(read_length_dict, blast_file, output_blast_file, pctid_thresh=85, cov_thresh=0.99):
    fblast=open(output_blast_file,"w")
    with open(blast_file) as infile:
        for i in infile:
            vec=i.split("\t")
            aln_len=vec[3]
            read_len=int(ldict[vec[0]])
            qhitlen=int(vec[7])-int(vec[6])
            cov=qhitlen/read_len
            pctid=float(vec[2])
            if cov>cov_thresh and pctid>pctid_thresh:
                fblast.write(i)
    fblast.close()

In [88]:
filter_reads(ldict, "./data/pb_subreads_vs_igenomes_concat.txt", "./data/pb_subreads_vs_igenomes_concat_filt2.txt", pctid_thresh=0.8,cov_thresh=0.9)

In [89]:
%%R
fb=read.table("./data/pb_subreads_vs_igenomes_concat_filt2.txt", sep="\t")

fb%>%group_by(V1, V2)%>%summarise(count=n())->read_ig_pairs


read_ig_pairs%>%nrow%>%print
read_ig_pairs$V1%>%unique%>%length%>%print
read_ig_pairs$V1%>%unique->rms
read_ig_pairs%>%group_by(count)%>%summarise(ct=n())
read_ig_pairs%>%group_by(V1)%>%summarise(ct=n())%>%group_by(ct)%>%summarise(ct1=n())->read_dist
colnames(read_dist)<-c("number_genomes_hit","read_count")
print(read_dist)
write.table(read_ig_pairs, "./data/concat_blast_read_ig_pairs_2.txt",sep="\t", quote=FALSE, row.names=FALSE)

[1] 65960
[1] 59023
Source: local data frame [11 x 2]

   number_genomes_hit read_count
                (int)      (int)
1                   1      54840
2                   2       3017
3                   3        548
4                   4        300
5                   5         96
6                   6         42
7                   7         28
8                   8         72
9                   9         68
10                 10          6
11                 11          6


In [91]:
%%R
groups<-read.table("/Users/jmb/Desktop/ViralFate/OrthoMCL/tSNE/tSNE_genome_clusters.txt", sep="\t", header=TRUE)
kkgroups<-read.table("/Users/jmb/Desktop/ViralFate/O")

head(groups)

     phage      tSNE1      tSNE2 dbscan_grps
1 1.003.O.  -42.87319  40.074449           1
2 1.004.O.  -72.00439  72.478087           2
3 1.005.O.   37.70535 -76.699446           3
4 1.007.O. -105.80774   9.436151           4
5 1.008.O.   20.10019  91.662298           5
6 1.009.O.   43.37092 -96.188388           3


In [96]:
%%R
phagefull<-read_ig_pairs$V2%>%unique
head(phagefull)

[1] Vibriophage_2.275.O._10N.286.54.E11 Vibriophage_3.058.O._10N.286.46.B8 
[3] Vibriophage_1.030.O._10N.222.55.F9  Vibriophage_1.165.O._10N.261.51.B7 
[5] Vibriophage_1.182.O._10N.286.46.E1  Vibriophage_1.228.O._10N.261.49.C1 
81 Levels: Vibriophage_1.007.O._10N.261.55.F9 ...


In [98]:
%Rpull phagefull
phageshort=[i.split("_")[1] for i in phagefull]
%Rpush phageshort

In [99]:
%%R
cbind.data.frame(phagefull, phageshort)->longshort
longshort$

                            phagefull phageshort
1 Vibriophage_2.275.O._10N.286.54.E11   2.275.O.
2  Vibriophage_3.058.O._10N.286.46.B8   3.058.O.
3  Vibriophage_1.030.O._10N.222.55.F9   1.030.O.
4  Vibriophage_1.165.O._10N.261.51.B7   1.165.O.
5  Vibriophage_1.182.O._10N.286.46.E1   1.182.O.
6  Vibriophage_1.228.O._10N.261.49.C1   1.228.O.


In [ ]:
%%R
